In [42]:
import tarfile
import numpy as np 
import os
import scipy.io as sio
#import rarfile
import torch
import matplotlib.pyplot as plt
#from hmmlearn import hmm
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import string
import numpy as np
import pandas as pd
import pickle as pkl
from collections import Counter
import random

In [4]:
import random
random.seed(0)
test_idx = random.sample(range(1,101), 20)
idx = list(set(range(1,101)) - set(test_idx))
train_idx = random.sample(idx, 70)
val_idx = list(set(idx) - set(train_idx))

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
dat_health = np.zeros([140*50, 81])
for i in range(70):
    t = str(train_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/Z_Healthy/c_Z'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health[i*50:(i+1)*50,:]=a
for i in range(70):
    t = str(train_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/O_Healthy/c_O'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health[(i+70)*50:(i+71)*50,:]=a
dat_inter = np.zeros([140*50, 81])
for i in range(70):
    t = str(train_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/F_Interictal/c_F'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter[i*50:(i+1)*50,:]=a
for i in range(70):
    t = str(train_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/N_Interictal/c_N'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter[(i+70)*50:(i+71)*50,:]=a
dat_ict = np.zeros([70*50, 81])
for i in range(70):
    t = str(train_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/S_Ictal/c_S'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_ict[i*50:(i+1)*50,:]=a

In [68]:
dat_health_t = np.zeros([20*50, 81])
for i in range(10):
    t = str(val_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/Z_Healthy/c_Z'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_t[i*50:(i+1)*50,:]=a
for i in range(10):
    t = str(val_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/O_Healthy/c_O'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_t[(i+10)*50:(i+11)*50,:]=a
dat_inter_t = np.zeros([20*50, 81])
for i in range(10):
    t = str(val_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/F_Interictal/c_F'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_t[i*50:(i+1)*50,:]=a
for i in range(10):
    t = str(val_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/N_Interictal/c_N'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_t[(i+10)*50:(i+11)*50,:]=a
dat_ict_t = np.zeros([10*50, 81])
for i in range(10):
    t = str(val_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/S_Ictal/c_S'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_ict_t[i*50:(i+1)*50,:]=a

In [70]:
dat_health = np.expand_dims(dat_health, 2)
dat_inter = np.expand_dims(dat_inter, 2)
dat_ict = np.expand_dims(dat_ict, 2)
dat_health_t = np.expand_dims(dat_health_t, 2)
dat_inter_t = np.expand_dims(dat_inter_t, 2)
dat_ict_t = np.expand_dims(dat_ict_t, 2)

In [71]:
dat_agg = np.concatenate([dat_health, dat_inter, dat_ict])
dat_lab = np.concatenate([[0]*140*50, [1]*140*50, [2]*70*50])
dat_agg_t = np.concatenate([dat_health_t, dat_inter_t, dat_ict_t])
dat_lab_t = np.concatenate([[0]*20*50, [1]*20*50, [2]*10*50])

In [38]:
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader, TensorDataset
class MyDataset(Dataset):
    def __init__(self, data_list, target_list):
        assert len(data_list) == len(target_list)
        self.data_list = data_list
        self.target_list = target_list
        
    def __getitem__(self, index):
        dat = self.data_list[index]
        label = self.target_list[index]
        
        return [dat, label]
    
    def __len__(self):
        return len(self.data_list)

In [45]:
def newsgroup_collate_func(batch):
    data_list = []
    label_list = []
    for datum in batch:
        label_list.append(datum[1])
        data_list.append(datum[0])
        
    x = torch.from_numpy(np.array(data_list)).float().to(device)
    y = torch.LongTensor(np.array(label_list)).to(device)
    return x, y

In [52]:
class RNN(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(RNN, self).__init__()
        
        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.rnn = nn.GRU(1, hidden_size, num_layers, batch_first=True)
        
        self.linear1 = nn.Linear(hidden_size, 3)

  
    def init_hidden(self, batch_size):
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size).float()

        return hidden

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        self.hidden = self.init_hidden(batch_size).to(device)

        _, self.hidden = self.rnn(x, self.hidden)
        final_hidden = self.hidden.view(1, 1, batch_size, self.hidden_size).contiguous().view(batch_size, -1)
        out = self.linear1(final_hidden)
        out = F.log_softmax(out, dim=1)
        return out

In [93]:
BATCH_SIZE = 64
train_dataset = MyDataset(dat_agg, dat_lab)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                batch_size=BATCH_SIZE,
                                collate_fn=newsgroup_collate_func,
                                shuffle=True)
val_dataset = MyDataset(dat_agg_t, dat_lab_t)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                batch_size=BATCH_SIZE,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)

In [147]:
hidden_size = 32
num_layers = 1
learning_rate = 0.001
model = RNN(hidden_size, num_layers).to(device)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [148]:
def do_train(model, dataloader, criterion, optimizer):
    model.train()
    train_loss = 0
    for data, labels in dataloader:
        optimizer.zero_grad()
        y_hat = model(data)
        loss = criterion(y_hat, labels.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * len(data) / len(dataloader.dataset)
    return train_loss

In [149]:
def do_eval(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    y_hat_ls = []
    y_ls = []
    with torch.no_grad():
        for data, labels in dataloader:
            y_hat = model(data)
            loss = criterion(y_hat, labels.long())
            y_hat_ls.append(y_hat)
            y_ls.append(labels)
            val_loss += loss.item() * len(data) / len(dataloader.dataset)
            
    return val_loss, torch.cat(y_hat_ls, dim=0), torch.cat(y_ls, dim=0)
def acc(model, dataloader, criterion):
    val_loss, pred,true = do_eval(
    model = model,
    dataloader = dataloader,
    criterion = criterion
    )
    return val_loss, (torch.exp(pred).max(1)[1] == true).float().mean().item()

In [150]:
    num_epochs = 100
    for epoch in range(num_epochs):
    #scheduler.step()
        loss = do_train(
            model=model, 
            criterion=criterion,
            dataloader=train_loader,
            optimizer=optimizer,
        )
        val_loss, val_acc = acc(model, val_loader, criterion)
        train_loss, train_acc = acc(model,train_loader, criterion)
        print('Epoch: [{}/{}], Train Loss: {}, Val Loss: {}, Val Acc: {}'.format( 
                           epoch+1, num_epochs, train_loss, val_loss, val_acc))

Epoch: [1/100], Train Loss: 1.037738937950135, Val Loss: 1.0466383695602415, Val Acc: 0.47279998660087585
Epoch: [2/100], Train Loss: 0.819499152197157, Val Loss: 0.9022808248519899, Val Acc: 0.6223999857902527
Epoch: [3/100], Train Loss: 0.7502304970060076, Val Loss: 0.8671154750823976, Val Acc: 0.6263999938964844
Epoch: [4/100], Train Loss: 0.69975699471065, Val Loss: 0.8475496284484865, Val Acc: 0.6499999761581421
Epoch: [5/100], Train Loss: 0.6660478685242788, Val Loss: 0.8382023738861085, Val Acc: 0.6535999774932861
Epoch: [6/100], Train Loss: 0.6475873438290184, Val Loss: 0.8094217693328858, Val Acc: 0.6583999991416931
Epoch: [7/100], Train Loss: 0.658844058806556, Val Loss: 0.884079648208618, Val Acc: 0.6431999802589417
Epoch: [8/100], Train Loss: 0.6095885052612849, Val Loss: 0.7964078742980956, Val Acc: 0.671999990940094
Epoch: [9/100], Train Loss: 0.5977741900580265, Val Loss: 0.7900070335388183, Val Acc: 0.6815999746322632
Epoch: [10/100], Train Loss: 0.595452929483141, Val 

KeyboardInterrupt: 

In [167]:
dat_agg_t = np.concatenate([dat_health_t, dat_inter_t, dat_ict_t])
dat_lab_t = np.concatenate([[0]*20*50, [1]*20*50, [2]*10*50])
t_dataset = MyDataset(dat_agg_test, dat_lab_test)
t_loader = torch.utils.data.DataLoader(dataset=t_dataset,
                                batch_size=50,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)

In [168]:
pred_lab = []
for x, y in t_loader:
    y_hat = model(x)
    pred = torch.exp(y_hat).max(1)[1].numpy()
    pred_lab.append(np.argmax([len(pred[pred==0]), len(pred[pred==1]), len(pred[pred==2])]))
rue_lab = np.concatenate([[0]*20,[1]*20,[2]*10])
print("validation accuracy: {}".format(len(true_lab[np.array(pred_lab)==true_lab])/50))

validation accuracy: 0.84


In [153]:
dat_health_test = np.zeros([40*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/Z_Healthy/c_Z'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_test[i*50:(i+1)*50,:]=a
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/O_Healthy/c_O'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_health_test[(i+20)*50:(i+21)*50,:]=a
dat_inter_test = np.zeros([40*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/F_Interictal/c_F'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_test[i*50:(i+1)*50,:]=a
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/N_Interictal/c_N'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_inter_test[(i+20)*50:(i+21)*50,:]=a
dat_ict_test = np.zeros([20*50, 81])
for i in range(20):
    t = str(test_idx[i]).zfill(3)
    a = np.loadtxt('C:/Users/gong/Documents/learning/TSA/project/Data/S_Ictal/c_S'+t+'.txt')
    a = a[:50*81].reshape([50, 81])
    dat_ict_test[i*50:(i+1)*50,:]=a

In [154]:
dat_health_test = np.expand_dims(dat_health_test, 2)
dat_inter_test = np.expand_dims(dat_inter_test, 2)
dat_ict_test = np.expand_dims(dat_ict_test, 2)

In [170]:
dat_agg_test = np.concatenate([dat_health_test, dat_inter_test, dat_ict_test])
dat_lab_test = np.concatenate([[0]*40*50, [1]*40*50, [2]*20*50])
test_dataset = MyDataset(dat_agg_test, dat_lab_test)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                batch_size=50,
                                collate_fn=newsgroup_collate_func,
                                shuffle=False)

In [171]:
pred_lab = []
for x, y in test_loader:
    y_hat = model(x)
    pred = torch.exp(y_hat).max(1)[1].numpy()
    pred_lab.append(np.argmax([len(pred[pred==0]), len(pred[pred==1]), len(pred[pred==2])]))
true_lab = np.concatenate([[0]*40,[1]*40,[2]*20])
print("test accuracy: {}".format(len(true_lab[np.array(pred_lab)==true_lab])/100))

test accuracy: 0.91
